In [85]:
from pandas_datareader import data as pdr
from ta.volatility import BollingerBands
from ta.trend import SMAIndicator
import yfinance as yf
import datetime as dt

yf.pdr_override ()

def get_ticker_df (stock, start = dt.datetime.now() - dt.timedelta(days=300), end = dt.datetime.now()):
    #stock = input ("Enter a stock ticker symbol: ")
    #stock = "TSLA"

    #start = dt.datetime (2019, 1, 1)
    #end = dt.datetime (2020, 1, 1)

    RECENT_HIGH_DAYS = 6

    df = pdr.get_data_yahoo (stock, start, end)

    # Fill in SMA

    col_name = "Adj Close"

    indicator_sma = SMAIndicator(close=df[col_name], window=5, fillna=True)

    df["SMA_5"] = indicator_sma.sma_indicator()


    indicator_sma = SMAIndicator(close=df[col_name], window=20, fillna=True)

    df["SMA_20"] = indicator_sma.sma_indicator()


    indicator_sma = SMAIndicator(close=df[col_name], window=50, fillna=True)

    df["SMA_50"] = indicator_sma.sma_indicator()

    indicator_sma = SMAIndicator(close=df[col_name], window=150, fillna=True)

    df["SMA_150"] = indicator_sma.sma_indicator()

    indicator_sma = SMAIndicator(close=df[col_name], window=200, fillna=True)

    df["SMA_200"] = indicator_sma.sma_indicator()

    #indicator_sma = SMAIndicator(close=df["Volume"], window=20, fillna=True)

    #df["Volume_SMA_20"] = indicator_sma.sma_indicator()

    df["Volume_SMA_20"] = df.rolling(window=20)['Volume'].mean()

    df["RVolume"] = df["Volume"] / df["Volume_SMA_20"]

    df["Closing Range"] = (df["Adj Close"] - df["Low"]) / (df["High"] - df["Low"]) 

    indicator_bb = BollingerBands(close=df["Adj Close"], window=15, window_dev=2)

    # Add Bollinger Bands features
    df['BB High'] = indicator_bb.bollinger_hband()
    df['BB Low'] = indicator_bb.bollinger_lband()

    #Previous Row data

    df['Prior Volume Change'] = round((df['Volume'] - df['Volume'].shift(1))/df['Volume'].shift(1), 2)

    high_vol = df["RVolume"] > 1.2
    high_dcr = df["Closing Range"] > .6
    low_dcr = df["Closing Range"] < .3
    high_prior_vol = df['Prior Volume Change'] > 0

    df["DCR Flag"] = 0
    
    df.loc[high_dcr, "DCR Flag"] = 1 
    df.loc[low_dcr, "DCR Flag"] = -1
    
    df["AccDist Flag"] = 0 
    df.loc[(high_vol | high_prior_vol) & high_dcr, "AccDist Flag"] = 1 
    df.loc[(high_vol | high_prior_vol) & low_dcr, "AccDist Flag"] = -1 

    return df


In [55]:
df_all = get_ticker_df("DOCN")
df = df_all.tail(1)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,SMA_5,SMA_20,SMA_50,SMA_150,SMA_200,Volume_SMA_20,RVolume,Closing Range,BB High,BB Low,AccDist
Date,,,,,,,,,,,,,,,,,
2021-10-18,89.0,92.459999,88.889999,91.660004,91.660004,928597,89.616,83.604,71.5002,54.803655,54.803655,1877409.85,0.494616,0.775912,93.476186,73.951814,NaN


In [96]:
start_date = dt.datetime.strptime("20210910", "%Y%m%d")
end_date = dt.datetime.strptime("20211021", "%Y%m%d")
base_df = get_ticker_df("SQ", start = start_date, end = end_date)

[*********************100%***********************]  1 of 1 completed


In [97]:
base_df["AccDist Flag"].tail(10)

Date
2021-10-07    1
2021-10-08    0
2021-10-11    0
2021-10-12    0
2021-10-13    0
2021-10-14   -1
2021-10-15    0
2021-10-18    1
2021-10-19    0
2021-10-20    0
Name: AccDist Flag, dtype: int64